## In this Notebook, I will demo how to use the FTXDataProcessor class to fetch historical data of perpetual and exipred futures data and calculate historical spreads. 

In [1]:
from keys import FTX_API_KEY, FTX_API_SECRET
from datetime import datetime
import time
import hmac
import pandas as pd
from requests import Request, Session, Response
from typing import Optional, Dict, Any, List
from utility_classes.historical_data_processor import FTXDataProcessor
import dateutil.parser as dp
import json

acc = FTXDataProcessor(api_key=FTX_API_KEY, api_secret=FTX_API_SECRET)


## get all tokens that have perpetual contracts

In [2]:

current_perp_futures = acc.get_all_perp_tickers()
print(current_perp_futures)


['1INCH', 'AAVE', 'ADA', 'ALGO', 'ALPHA', 'ALT', 'AMPL', 'AR', 'ATOM', 'AVAX', 'BADGER', 'BAL', 'BAND', 'BAO', 'BAT', 'BCH', 'BNB', 'BNT', 'BRZ', 'BSV', 'BTC', 'BTMX', 'BTT', 'CAKE', 'CHZ', 'COMP', 'CREAM', 'CRV', 'CUSDT', 'DASH', 'DEFI', 'DMG', 'DOGE', 'DOT', 'DRGN', 'EGLD', 'ENJ', 'EOS', 'ETC', 'ETH', 'EXCH', 'FIDA', 'FIL', 'FLM', 'FLOW', 'FTM', 'FTT', 'GRT', 'HNT', 'HOLY', 'HT', 'KAVA', 'KIN', 'KNC', 'KSM', 'LEO', 'LINA', 'LINK', 'LRC', 'LTC', 'LUNA', 'MAPS', 'MATIC', 'MID', 'MKR', 'MTA', 'NEAR', 'NEO', 'OKB', 'OMG', 'ONT', 'OXY', 'PAXG', 'PERP', 'PRIV', 'QTUM', 'RAY', 'REEF', 'REN', 'ROOK', 'RSR', 'RUNE', 'SECO', 'SHIT', 'SNX', 'SOL', 'SRM', 'SUSHI', 'SXP', 'THETA', 'TOMO', 'TRU', 'TRX', 'TRYB', 'UNI', 'UNISWAP', 'USDT', 'VET', 'WAVES', 'XAUT', 'XLM', 'XMR', 'XRP', 'XTZ', 'YFI', 'YFII', 'ZEC', 'ZIL', 'ZRX']


## getting symbols for all expired futures

In [3]:
expired_futures = acc._get('expired_futures')
for ticker in expired_futures: 
    print(ticker['name'], ticker['expiryDescription'])


mber 2020
AMZN-1225 December 2020
UBER-1225 December 2020
NVDA-1225 December 2020
EOS-1225 December 2020
BTMX-1225 December 2020
LINK-1225 December 2020
SOL-1225 December 2020
MID-1225 December 2020
DOT-1225 December 2020
DMG-1225 December 2020
BCH-1225 December 2020
SUSHI-1225 December 2020
TWTR-1225 December 2020
BNB-1225 December 2020
TLRY-1225 December 2020
SPY-1225 December 2020
BABA-1225 December 2020
ATOM-1225 December 2020
MATIC-1225 December 2020
OKB-1225 December 2020
BAL-1225 December 2020
SQ-1225 December 2020
TOMO-1225 December 2020
MRNA-1225 December 2020
BSV-1225 December 2020
RUNE-1225 December 2020
ABNB-1225 December 2020
CREAM-1225 December 2020
SHIT-1225 December 2020
GOOGL-1225 December 2020
BTC-HASH-2020Q4 2020 Q4
DRGN-1225 December 2020
USDT-1225 December 2020
TRYB-1225 December 2020
BTC-1225 December 2020
BTC-MOVE-1224 2020-12-24
BTC-MOVE-1223 2020-12-23
BTC-MOVE-1222 2020-12-22
BTC-MOVE-1221 2020-12-21
BTC-MOVE-1220 2020-12-20
BTC-MOVE-1219 2020-12-19
BTC-MOVE-1

## To calculate spreads, I need to make sure that all the futures fetched have perpetual contract counterparts

In [11]:
expired_futures=acc.get_all_expired_futures_that_have_perps()
expired_futures.sort()
for sym in expired_futures:
    print(sym)

AAVE-1225
ADA-0626
ADA-0925
ADA-1225
ADA-20191227
ADA-20200327
ALGO-0626
ALGO-0925
ALGO-1225
ALGO-20190927
ALGO-20191227
ALGO-20200327
ALT-0626
ALT-0925
ALT-1225
ALT-20190927
ALT-20191227
ALT-20200327
ATOM-0626
ATOM-0925
ATOM-1225
ATOM-20191227
ATOM-20200327
AVAX-0925
AVAX-1225
BAL-0626
BAL-0925
BAL-1225
BCH-0626
BCH-0925
BCH-1225
BCH-20190927
BCH-20191227
BCH-20200327
BNB-0626
BNB-0925
BNB-1225
BNB-20190927
BNB-20191227
BNB-20200327
BRZ-0626
BRZ-0925
BRZ-1225
BSV-0626
BSV-0925
BSV-1225
BSV-20190927
BSV-20191227
BSV-20200327
BTC-0626
BTC-0925
BTC-1225
BTC-20190927
BTC-20191227
BTC-20200327
BTMX-0626
BTMX-0925
BTMX-1225
BTMX-20191227
BTMX-20200327
COMP-0626
COMP-0925
COMP-1225
CREAM-0925
CREAM-1225
CUSDT-0626
CUSDT-0925
DEFI-0626
DEFI-0925
DEFI-1225
DMG-0925
DMG-1225
DOGE-0626
DOGE-0925
DOGE-1225
DOGE-20190927
DOGE-20191227
DOGE-20200327
DOT-0925
DOT-1225
DOTPRESPLIT-0925
DOTPRESPLIT-PERP
DRGN-0626
DRGN-0925
DRGN-1225
DRGN-20191227
DRGN-20200327
EOS-0626
EOS-0925
EOS-1225
EOS-20190927
E

## all past futures expiration dates format
### to get past data requires correct date input 
### *only futures data after 20190628 is available on FTX

In [65]:
acc.get_expired_futures_dates()


expiration date: 0326 | Date March 2021
expiration date: 1225 | Date December 2020
expiration date: 0925 | Date September 2020
expiration date: 0626 | Date June 2020
expiration date: 20200327 | Date March 2020
expiration date: 20191227 | Date December 2019
expiration date: 20190927 | Date September 2019
expiration date: 20190628 | Date June 2019
expiration date: 20190329 | Date March 2019


## To get an expired futures contract given specific expired date
### *BTC futures have longer periods than alts


In [80]:
alt_hist_futures = acc.get_expired_futures_OHCL(market = 'ETH-0925', resolution = 3600)
btc_hist_futures = acc.get_expired_futures_OHCL(market = 'BTC-0925', resolution = 3600)
print(alt_hist_futures)
print(btc_hist_futures)

Adding 2613 candles with start time 2020-09-24 23:00:00
Adding 1 candles with start time 2020-06-08 02:00:00
Adding 4864 candles with start time 2020-09-24 23:00:00
Adding 1 candles with start time 2020-03-06 06:00:00
                      timestamp     open    high     low   close  \
0     2020-06-08T06:00:00+00:00  248.045  248.26  247.36  247.54   
1     2020-06-08T07:00:00+00:00  247.540  247.96  247.05  247.36   
2     2020-06-08T08:00:00+00:00  247.360  247.87  246.13  246.31   
3     2020-06-08T09:00:00+00:00  246.310  248.19  246.31  247.82   
4     2020-06-08T10:00:00+00:00  247.850  248.05  247.00  247.09   
...                         ...      ...     ...     ...     ...   
2608  2020-09-24T22:00:00+00:00  345.860  352.78  345.35  350.99   
2609  2020-09-24T23:00:00+00:00  350.990  351.37  348.47  349.27   
2610  2020-09-25T00:00:00+00:00  349.270  350.31  347.09  347.92   
2611  2020-09-25T01:00:00+00:00  347.920  348.64  345.86  346.36   
2612  2020-09-25T02:00:00+00:00  3

## get all historical perp contract funding rate

In [3]:
acc.get_all_historical_funding_rates(path='/home/harry/trading_algo/crypto_trading_researches/strategy_backtests/historical_data/historical_funding')


:00
Adding 500 funding data with start time 2020-08-25 23:00:00
Adding 500 funding data with start time 2020-08-05 04:00:00
Adding 225 funding data with start time 2020-07-26 20:00:00
Adding 1 funding data with start time 2020-07-26 20:00:00
Adding 500 funding data with start time 2021-03-01 01:00:00
Adding 448 funding data with start time 2021-02-10 10:00:00
Adding 1 funding data with start time 2021-02-10 10:00:00
Adding 500 funding data with start time 2021-03-01 01:00:00
Adding 500 funding data with start time 2021-02-08 06:00:00
Adding 500 funding data with start time 2021-01-18 11:00:00
Adding 500 funding data with start time 2020-12-28 16:00:00
Adding 500 funding data with start time 2020-12-07 21:00:00
Adding 500 funding data with start time 2020-11-17 02:00:00
Adding 500 funding data with start time 2020-10-27 08:00:00
Adding 500 funding data with start time 2020-10-06 13:00:00
Adding 500 funding data with start time 2020-09-15 18:00:00
Adding 361 funding data with start time 

## fetching all expired futures and write them into csv


In [2]:
acc.write_all_PERPs_OHCL(path='/home/harry/trading_algo/crypto_trading_researches/strategy_backtests/historical_data/all_perps')


art time 2020-01-28 15:43:00
Adding 5001 candles with start time 2020-01-25 04:23:00
Adding 5001 candles with start time 2020-01-21 17:03:00
Adding 5001 candles with start time 2020-01-18 05:43:00
Adding 5001 candles with start time 2020-01-14 18:23:00
Adding 5001 candles with start time 2020-01-11 07:03:00
Adding 5001 candles with start time 2020-01-07 19:43:00
Adding 5001 candles with start time 2020-01-04 08:23:00
Adding 5001 candles with start time 2019-12-31 21:03:00
Adding 5001 candles with start time 2019-12-28 09:43:00
Adding 5001 candles with start time 2019-12-24 22:23:00
Adding 5001 candles with start time 2019-12-21 11:03:00
Adding 5001 candles with start time 2019-12-17 23:43:00
Adding 5001 candles with start time 2019-12-14 12:23:00
Adding 5001 candles with start time 2019-12-11 01:03:00
Adding 5001 candles with start time 2019-12-07 13:43:00
Adding 5001 candles with start time 2019-12-04 02:23:00
Adding 5001 candles with start time 2019-11-30 15:03:00
Adding 5001 candles

In [2]:
acc.write_all_expired_futures_OHCL(path='/home/harry/trading_algo/crypto_trading_researches/strategy_backtests/historical_data/expired_futures_data', resolution=60)


rt time 2019-11-29 06:23:00
Adding 5001 candles with start time 2019-09-26 23:00:00
Adding 5001 candles with start time 2019-09-23 11:40:00
Adding 5001 candles with start time 2019-09-20 00:20:00
Adding 5001 candles with start time 2019-09-16 13:00:00
Adding 5001 candles with start time 2019-09-13 01:40:00
Adding 5001 candles with start time 2019-09-09 14:20:00
Adding 4997 candles with start time 2019-09-06 03:00:00
Adding 5001 candles with start time 2019-09-02 15:40:00
Adding 5001 candles with start time 2019-08-30 04:20:00
Adding 5001 candles with start time 2019-08-26 17:00:00
Adding 5001 candles with start time 2019-08-23 05:40:00
Adding 5001 candles with start time 2019-08-19 18:20:00
Adding 4994 candles with start time 2019-08-16 07:00:00
Adding 5001 candles with start time 2019-08-12 19:40:00
Adding 5001 candles with start time 2019-08-09 08:20:00
Adding 5001 candles with start time 2019-08-05 21:00:00
Adding 5001 candles with start time 2019-08-02 09:40:00
Adding 4999 candles 

## writing spreads data for all perp-future pairs

In [3]:
acc.write_all_spreads(
    perp_folder_path='/home/harry/trading_algo/crypto_trading_researches/strategy_backtests/historical_data/all_perps',
    futures_folder_path='/home/harry/trading_algo/crypto_trading_researches/strategy_backtests/historical_data/expired_futures_data',
    output_path='/home/harry/trading_algo/crypto_trading_researches/strategy_backtests/historical_data/all_spreads')

Writing spread for 1INCH-0326
Writing spread for AAVE-0326
Writing spread for AAVE-1225
Writing spread for ADA-0326
Writing spread for ADA-0626
Writing spread for ADA-0925
Writing spread for ADA-1225
Writing spread for ADA-20191227
Writing spread for ADA-20200327
Writing spread for ALGO-0326
Writing spread for ALGO-0626
Writing spread for ALGO-0925
Writing spread for ALGO-1225
Writing spread for ALGO-20190927
Writing spread for ALGO-20191227
Writing spread for ALGO-20200327
Writing spread for ALT-0326
Writing spread for ALT-0626
Writing spread for ALT-0925
Writing spread for ALT-1225
Writing spread for ALT-20190927
Writing spread for ALT-20191227
Writing spread for ALT-20200327
Writing spread for ATOM-0326
Writing spread for ATOM-0626
Writing spread for ATOM-0925
Writing spread for ATOM-1225
Writing spread for ATOM-20191227
Writing spread for ATOM-20200327
Writing spread for AVAX-0326
Writing spread for AVAX-0925
Writing spread for AVAX-1225
Writing spread for BAL-0326
Writing spread f